In [1]:
import os

## NEW
- skip threading and FastRelax

In [2]:
import os
num_gpus=5 #How many gpus to use

scaffold_dir="scaffolds/scaffolds_V5/"
separate_scaffold_folders=True

prefix="CD4-dom2-sample44-noise-40x"
target_pdb="target_pdb/c4-dom2-1cdh.pdb"
hotspots='ppi.hotspot_res=[A170,A1130,A165,A157,A135]'

num_of_diffusions=40 # Number of RF diffusions to do per scaffold
num_seqs=50 # How many MPNN sequences to generate per RF diffusion
num_filtered_seqs=50 # How many of the MPNN generated sequences per RF diffusion to keep for AF2

rfdiffusion_config = [
  "diffuser.T=50",
  "'potentials.guiding_potentials=[\"type:binder_ROG,weight:3\"]'", # "type:binder_distance_ReLU",\"type:interface_ncontacts\"
  "potentials.guide_scale=2",
  "potentials.guide_decay=\"quadratic\"",
  "denoiser.noise_scale_ca=0.5",
  "denoiser.noise_scale_frame=0.5"]

if "[A" in hotspots:
    rfdiffusion_config.append(hotspots)
rfdiffusion_argument_string= "+".join(rfdiffusion_config)
# Other parameters
num_recycles=3 # AF2 recycles
sampling_temp=0.0001 # ProteinMPNN sampling temperature

def get_subdirectories(folder_path):
    subdirectories = []
    for entry in os.scandir(folder_path):
        if entry.is_dir():
            subdirectories.append(entry.path)
    return subdirectories

if separate_scaffold_folders:
    scaffold_list=get_subdirectories(scaffold_dir)
    scaffolds=len(scaffold_list)
    print(scaffold_list)
print(f"JOB: {prefix}")
print(f"TARGET: {target_pdb}")
print(f"HOTSPOTS: {hotspots}")

def directory_exists(directory):
    return os.path.isdir(directory)

directory = f"output/{prefix}"
if directory_exists(directory):
    print("""\033[1;31;40m#####################
PREFIX ALREADY EXISTS
#####################""")
else:
    print("Prefix does not exist yet")
gpus_in_use = !squeue | grep -c 'gpu.*compute\|compute.*gpu'
print(f"\033[0mNumber of gpus already in use: {gpus_in_use[0]}/50", end="")

This will generate 1000 designs per array job! In total 240000 designs (4800 diffusions)!


## Runs slurm array script

In [5]:
import subprocess
os.makedirs(directory, exist_ok=True)
if separate_scaffold_folders:
    if num_gpus%scaffolds==0:   #If number of gpus is divisible by the number of scaffolds, make it so that each scaffold has its own gpu(s)
        gpus_per_scaffold = num_gpus//scaffolds
        diffusions_per_gpu = num_of_diffusions//gpus_per_scaffold
        for i in range(scaffolds):
            # Setup arguments
            scaffold_dir=scaffold_list[i]
            script_arguments=f"{diffusions_per_gpu} {num_seqs} {num_filtered_seqs} {num_recycles} {sampling_temp} {scaffold_dir} {prefix} {target_pdb} {rfdiffusion_argument_string}"
            bash_arguments=f"--output={directory}/slurm-%A_%a.out --job-name={prefix}_binder_docking --array=1-{gpus_per_scaffold}"
            
            command = f"sbatch {bash_arguments} helper_scripts/binder_dock_nothread.sh {script_arguments}"

            # Run the array bash script
            print(f"Running {command}")
            subprocess.run(command, shell=True)
    else:
        num_per_gpu=num_of_diffusions//num_gpus
        scaffold_string = ','.join(scaffold_list)   #Converts list of scaffold directories into a string for parsing in shell script
        script_arguments=f"{num_per_gpu} {num_seqs} {num_filtered_seqs} {num_recycles} {sampling_temp} {scaffold_string} {prefix} {target_pdb} {rfdiffusion_argument_string}"
        bash_arguments=f"--output={directory}/slurm-%A_%a.out --job-name={prefix}_binder_docking --array=1-{num_gpus}"
            
        command = f"sbatch {bash_arguments} helper_scripts/binder_dock_nothread.sh {script_arguments}"

        # Run the array bash script
        print(f"Running {command}")
        subprocess.run(command, shell=True)

else:
    script_arguments=f"{num_of_diffusions} {num_seqs} {num_filtered_seqs} {num_recycles} {sampling_temp} {scaffold_dir} {prefix} {target_pdb} {rfdiffusion_argument_string}"
    bash_arguments=f"--output={directory}/slurm-%A_%a.out --job-name={prefix}_binder_docking --array=1-{num_gpus}"
    
    command = f"sbatch {bash_arguments} helper_scripts/binder_dock_nothread.sh {script_arguments}"

    # Run the array bash script
    print(f"Running {command}")
    subprocess.run(command, shell=True)

#print(f"command: {command}")

Running sbatch --job-name=AH2_helix_binder_docking --array=1-40%1 helper_scripts/binder_dock.sh 20 50 50 3 0.0001 ../../scaffolds/scaffolds_V4/27_3H AH2_helix /home/tsatler/RFdif/ClusterProteinDesign/scripts/binder_design/examples/AH2_helix.pdb ppi.hotspot_res=[A59,A61,A64,A66]
Submitted batch job 328456
Running sbatch --job-name=AH2_helix_binder_docking --array=1-40%1 helper_scripts/binder_dock.sh 20 50 50 3 0.0001 ../../scaffolds/scaffolds_V4/60_3Hs AH2_helix /home/tsatler/RFdif/ClusterProteinDesign/scripts/binder_design/examples/AH2_helix.pdb ppi.hotspot_res=[A59,A61,A64,A66]
Submitted batch job 328457
Running sbatch --job-name=AH2_helix_binder_docking --array=1-40%1 helper_scripts/binder_dock.sh 20 50 50 3 0.0001 ../../scaffolds/scaffolds_V4/4H_9 AH2_helix /home/tsatler/RFdif/ClusterProteinDesign/scripts/binder_design/examples/AH2_helix.pdb ppi.hotspot_res=[A59,A61,A64,A66]
Submitted batch job 328458
Running sbatch --job-name=AH2_helix_binder_docking --array=1-40%1 helper_scripts/b

In [13]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
   328447_[8-56%8]       gpu predict_  tsatler PD       0:00      1 (JobArrayTaskLimit)
   328456_[2-40%1]       gpu AH2_heli  tsatler PD       0:00      1 (JobArrayTaskLimit)
   328457_[2-40%1]       gpu AH2_heli  tsatler PD       0:00      1 (JobArrayTaskLimit)
   328458_[2-40%1]       gpu AH2_heli  tsatler PD       0:00      1 (JobArrayTaskLimit)
   328459_[2-40%1]       gpu AH2_heli  tsatler PD       0:00      1 (JobArrayTaskLimit)
   328460_[2-40%1]       gpu AH2_heli  tsatler PD       0:00      1 (JobArrayTaskLimit)
   328461_[2-40%1]       gpu AH2_heli  tsatler PD       0:00      1 (JobArrayTaskLimit)
          328447_0       gpu predict_  tsatler  R       2:08      1 compute-0-12
          328447_1       gpu predict_  tsatler  R       2:08      1 compute-0-12
          328447_2       gpu predict_  tsatler  R       2:08      1 compute-6-1
          328447_3       gpu predict_  tsatler  R       2